#  쿼리 확장 (Query Expansion)

--- 

## 1. 개념 이해

### 1.1 쿼리 확장이란?

- 쿼리 확장(Query Expansion)은 사용자의 원본 질문을 보다 효과적인 검색을 위해 변형하거나 확장하는 기법입니다. 
- RAG 시스템에서 검색의 품질은 최종 답변의 품질을 좌우하는 핵심 요소이므로, 적절한 쿼리 확장 기법을 통해 검색 성능을 크게 향상시킬 수 있습니다.

### 1.2 쿼리 확장의 필요성

#### 🔍 **검색의 한계**
- **어휘 불일치 문제**: 사용자 질문과 문서 간 용어 차이
- **모호한 질문**: 불명확하거나 맥락이 부족한 쿼리
- **복잡한 질문**: 여러 하위 질문을 포함하는 복합 질문
- **의도 파악 어려움**: 사용자의 실제 의도와 표면적 질문의 차이

#### 🎯 **해결 방향**
- **의미론적 확장**: 동의어, 관련 개념 추가
- **구조적 분해**: 복잡한 질문을 단순한 하위 질문으로 분해
- **맥락적 확장**: 배경 지식과 일반적 개념 추가
- **가상 문서 생성**: 이상적 답변을 통한 검색 개선

### 1.3 쿼리 확장 방법론 분류

| 방법론 | 핵심 아이디어 | 장점 | 단점 | 적용 상황 |
|--------|---------------|------|------|-----------|
| **Query Reformulation** | LLM으로 질문 재작성 | 구현 간단, 즉시 적용 | 단일 변형만 생성 | 일반적인 질문 개선 |
| **Multi Query** | 다양한 관점의 질문 생성 | 검색 다양성 증가 | 계산 비용 증가 | 모호한 질문 처리 |
| **Decomposition** | 복잡한 질문을 하위 질문으로 분해 | 체계적 접근 | 분해 정확도 의존 | 복합적 질문 |
| **Step-Back Prompting** | 일반적 맥락에서 구체적 답변으로 | 포괄적 이해 | 추가 검색 필요 | 전문적/복잡한 질문 |
| **HyDE** | 가상 답변 문서 생성 | 의미적 정렬 우수 | 환각 위험 | Zero-shot 상황 |

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
import json
from pprint import pprint
from typing import List, Dict, Any
import pandas as pd
import numpy as np

# LangChain 핵심
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, BaseOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 검색 및 벡터 저장
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


`(4) 벡터스토어 로드`

In [4]:
def initialize_vector_store(embeddings = OpenAIEmbeddings(model="text-embedding-3-small"), collection_name="hybrid_search_db", persist_directory = "./local_chroma_db"):
    """
    기존 벡터 저장소를 로드하거나 새로 생성
    
    Returns:
        Chroma: 벡터 저장소 객체
    """
    try:
        
        # 기존 벡터 저장소 로드 시도
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory,
        )
        
        doc_count = vector_store._collection.count()
        if doc_count > 0:
            print(f"✅ 기존 벡터 저장소 로드: {doc_count}개 문서")
            return vector_store
        else:
            print("⚠️ 빈 벡터 저장소입니다. 데이터를 추가해주세요.")
            return vector_store
            
    except Exception as e:
        print(f"❌ 벡터 저장소 로드 실패: {e}")
        return None

# 벡터 저장소 초기화
chroma_db = initialize_vector_store()

✅ 기존 벡터 저장소 로드: 39개 문서


`(5) 백터 검색기 생성`

In [5]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments

### 1) **Query Reformulation** 

- **Query Reformulation**은 **LLM**을 활용해 원본 질문을 다양한 형태로 재구성
- **동의어 확장**과 **키워드 추가**를 통해 검색 쿼리의 범위를 확장
- 모호한 질문을 **명확하게 구체화**하여 검색 정확도 향상
- 하나의 질문에 대해 **다양한 변형 쿼리**를 생성하여 검색 커버리지 확대

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_rewrite.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2305.14283

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 쿼리 리포뮬레이션을 위한 프롬프트 템플릿 정의
reformulation_template = """다음 질문을 검색 성능을 향상시키기 위해 다시 작성해주세요:
[질문]
{question}

다음 방식으로 질문을 재작성하세요:
1. 동의어 추가
2. 더 구체적인 키워드 포함
3. 관련된 개념 확장

[재작성된 질문]
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(reformulation_template)

# LLM 모델 초기화
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 쿼리 리포뮬레이션 체인 생성
reformulation_chain = prompt | llm | StrOutputParser()

# 체인 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
reformulated_query = reformulation_chain.invoke({"question": query})

print(f"쿼리: {query}")
pprint(f"리포뮬레이션된 쿼리: \n{reformulated_query}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
('리포뮬레이션된 쿼리: \n'
 '[재작성된 질문]  \n'
 '리비안(Rivian)의 핵심 사업 경쟁력과 차별화된 강점은 무엇이며, 전기차 시장 내에서의 기술 혁신, 공급망 관리, 그리고 지속 가능성 '
 '전략은 어떻게 작용하고 있나요?')


In [7]:
# 리포뮬레이션된 쿼리로 검색
retrieved_docs = chroma_k_retriever.invoke(reformulated_query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivian은 IPO를 통해 135억 달러를 조달하여 상장 회사가 됨.
- Ford와 Rivian은 전기 자동차 공동 개발 계획을 취소.
- 2022년 3월, Rivian은 부품 비용으로 인해 R1T 및 R1S의 가격 인상을 발표했으며, 이후 소급 적용에 대해 사과.
- 2022년 7월, Rivian은 6%의 인력 감축을 발표.
- 2022년 9월, 유럽에서 전기 밴을 생산하기 위해 Mercedes-Benz Group과 합작 투자를 발표했지만 나중에 취소됨.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity In

In [8]:
# Runnable 객체로 변환하여 검색기 생성 (LCEL)
reformulation_retriever = reformulation_chain | chroma_k_retriever

# 쿼리 리포뮬레이션 검색기 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = reformulation_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송**
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivian은 IPO를 통해 135억 달러를 조달하여 상장 회사가 됨.
- Ford와 Rivian은 전기 자동차 공동 개발 계획을 취소.
- 2022년 3월, Rivian은 부품 비용으로 

In [9]:
def load_evaluation_dataset(file_path):
    """
    평가 데이터셋 로드
    
    Args:
        file_path (str): 평가 데이터 파일 경로
    
    Returns:
        pandas.DataFrame: 평가 데이터셋
    """
    try:
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            raise ValueError("지원하지 않는 파일 형식")
        
        print(f"✅ 평가 데이터셋 로드: {len(df)}개 질문")
        return df
    except Exception as e:
        print(f"❌ 데이터셋 로드 실패: {e}")
        return None

# 평가 데이터셋 로드

eval_df = load_evaluation_dataset("./data/synthetic_testset.csv")
eval_df.head(3)

✅ 평가 데이터셋 로드: 50개 질문


,user_input,reference_contexts,reference,synthesizer_name
0,"Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?","['Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...",single_hop_specifc_query_synthesizer
1,168억 미국 달러는 리브니의 무슨 항목에 해당합니까?,['- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- ...,168억 미국 달러는 2023년 기준 리브니의 총 자산에 해당합니다.,single_hop_specifc_query_synthesizer
2,2023년 기준 Rivian의 직원 수는 몇 명입니까?,['- **총 자본 (2023):** 91억 4천만 미국 달러\n- **직원 수 (...,"2023년 12월 기준 Rivian의 직원 수는 16,790명입니다.",single_hop_specifc_query_synthesizer


In [10]:
def prepare_evaluation_data(df):
    """
    평가 데이터 전처리
    
    Args:
        df (pandas.DataFrame): 원본 데이터프레임
    
    Returns:
        tuple: (질문 리스트, 정답 문서 리스트)
    """
    questions = df['user_input'].tolist()
    
    # 정답 문서 파싱
    reference_contexts = []
    for contexts in df['reference_contexts']:
        if isinstance(contexts, str):
            # 문자열을 리스트로 변환
            context_list = eval(contexts)
        else:
            context_list = contexts
        
        # Document 객체로 변환
        docs = [Document(page_content=ctx) for ctx in context_list]
        reference_contexts.append(docs)
    
    return questions, reference_contexts

# 평가 데이터 전처리
questions, reference_contexts = prepare_evaluation_data(eval_df)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[:3], reference_contexts[:3])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[-3:], reference_contexts[-3:])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)


[질문 1]
질문: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?
정답 문서: 1개
  [1] 내용: Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업...
--------------------------------------------------

[질문 2]
질문: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?
정답 문서: 1개
  [1] 내용: - **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** ...
--------------------------------------------------

[질문 3]
질문: 2023년 기준 Rivian의 직원 수는 몇 명입니까?
정답 문서: 1개
  [1] 내용: - **총 자본 (2023):** 91억 4천만 미국 달러
- **직원 수 (2023년 1...
--------------------------------------------------

[질문 1]
질문: Tesla의 Bitcoin 투자와 NASDAQ-100 상장, 그리고 2020년 주가 상승은 어떻게 연결되며, 이 세 가지가 Tesla의 전기차 사업 성장에 어떤 영향을 미쳤습니까?
정답 문서: 3개
  [1] 내용: <1-hop>

2021년 초, Tesla는 Bitcoin에 15억 달러를 투자하고 환경 ...
  [2] 내용: <2-hop>

Tesla는 2012년 6월 Model S 고급 세단을 출시했습니다. Mo...
  [3] 내용: <3-hop>

2019년 7월부터 2020년 6월까지 Tesla는 4분기 연속 흑자를 보...
--------------------------------------------------

[질문 2]
질문: Tesla의 Autopilot driver assistance system과 향후 출시 예정인 Autonomous driving technology 차량들은 무엇이며, 각각의 특징

In [11]:
# ranx-k 라이브러리 사용해서 검색 결과 평가
from ranx_k.evaluation import evaluate_with_ranx_similarity

# ranx-k 평가 실행 (rouge 점수가 높은 경우) -> 문자열 유사도 기반 평가
chroma_results = evaluate_with_ranx_similarity(
    retriever=chroma_k_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

c:\Users\ys\ktds-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Starting similarity-based ranx evaluation | 유사도 기반 ranx 평가 시작
   Method | 방법: kiwi_rouge, Threshold | 임계값: 0.8, Mode | 모드: reference_based
   📊 Reference-based mode: Will calculate proper recall metrics | 참조 기반 모드: 정확한 재현율 계산

🔍 Question | 질문 1: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.800
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 질문 2: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.930
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 

ranx similarity evaluation | ranx 유사도 평가: 100%|██████████| 50/50 [00:20<00:00,  2.35it/s]



📊 Similarity-based ranx evaluation results | 유사도 기반 ranx 평가 결과 (kiwi_rouge):
  hit_rate@5: 0.860
  ndcg@5: 0.749
  map@5: 0.702
  mrr: 0.718

📈 Analysis information | 분석 정보:
  Total queries | 총 질문 수: 50
  Total retrieved docs | 총 검색 문서 수: 250
  Avg docs per query | 질문당 평균 검색 문서: 5.0
  Total relevant docs | 관련 문서 총 개수: 76
  Avg relevant per query | 질문당 평균 관련 문서: 1.5
  Threshold used | 사용된 임계값: 0.8
  Overall recall | 전체 재현율: 0.789 (60/76)


In [12]:
reformulation_results = evaluate_with_ranx_similarity(
    retriever=reformulation_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

🔍 Starting similarity-based ranx evaluation | 유사도 기반 ranx 평가 시작
   Method | 방법: kiwi_rouge, Threshold | 임계값: 0.8, Mode | 모드: reference_based
   📊 Reference-based mode: Will calculate proper recall metrics | 참조 기반 모드: 정확한 재현율 계산

🔍 Question | 질문 1: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.800
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 질문 2: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.930
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 

ranx similarity evaluation | ranx 유사도 평가: 100%|██████████| 50/50 [01:54<00:00,  2.43s/it]


📊 Similarity-based ranx evaluation results | 유사도 기반 ranx 평가 결과 (kiwi_rouge):
  hit_rate@5: 0.900
  ndcg@5: 0.714
  map@5: 0.640
  mrr: 0.661

📈 Analysis information | 분석 정보:
  Total queries | 총 질문 수: 50
  Total retrieved docs | 총 검색 문서 수: 250
  Avg docs per query | 질문당 평균 검색 문서: 5.0
  Total relevant docs | 관련 문서 총 개수: 76
  Avg relevant per query | 질문당 평균 관련 문서: 1.5
  Threshold used | 사용된 임계값: 0.8
  Overall recall | 전체 재현율: 0.776 (59/76)


### 2) **Multi Query** 

- **Multi Query**는 **Retriever의 LLM**을 사용해 단일 질문을 다수의 쿼리로 확장
- 원본 질문에 대해 **다양한 관점**과 **표현 방식**으로 쿼리 자동 생성
- **LLM의 생성 능력**을 활용해 검색 범위를 자연스럽게 확장
- 검색의 **다양성**과 **포괄성**이 향상되어 관련 문서 검색 확률 증가


<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/multi-query.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>

[출처] https://arxiv.org/abs/2411.13154

`(1) MultiQueryRetriever 활용`

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [13]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 (멀티 쿼리 생성용)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.7,
)

# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = multi_query_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
## 파트너

Tesla는 Panasonic과 파트너십을 맺고 있으며 리튬 공급에 대한 장기 계약을 맺고 있습니다. 이전 파트너로는 Daimler와 Toyota가 있습니다.

## 소송 및 논란

Tesla는 성희롱, 노동 분쟁, 사기 혐의, 대리점 분쟁, 지적 재산권, 환경 위반, 재산 피해, 인종 차별, COVID-19 팬데믹 대응 및 수리 권리와 관련된 소송 및 논란에 직면했습니다.

## 비판

Tesla는 데이터 개인 정보 보호, 공매도자, 지연, 차량 제품 문제, 화재, Autopilot 충돌, 소프트웨어 해킹, 가상 제동 및 주행 거리 성능과 관련된 비판에 직면했습니다.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

`(2) Custom Prompt 활용`

In [14]:
from typing import List

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 출력 파서: LLM 결과를 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        """Split the text into lines and remove empty lines."""
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    

# 쿼리 생성 프롬프트
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Generate three different versions of the given user question to retrieve relevant documents from a vector database. The goal is to reframe the question from various perspectives to overcome limitations of distance-based similarity search.

    The generated questions should have the following characteristics:
    1. Maintain the core intent of the original question but use different expressions or viewpoints.
    2. Include synonyms or related concepts where possible.
    3. Slightly broaden or narrow the scope of the question to potentially include diverse relevant information.

    Write each question on a new line and include only the questions.

    [Original question]
    {question}
    
    [Alternative questions]
    """,
)

# 멀티쿼리 체인 구성
multiquery_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = multiquery_chain.invoke({"question": query})

print("생성된 대안 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 대안 질문들:
1. 리비안이 시장에서 경쟁 우위를 확보할 수 있는 핵심 요인은 무엇인가요?
2. 리비안의 비즈니스 강점과 성공 요소는 어떤 부분에 기반하나요?
3. 리비안이 산업 내에서 뛰어난 경쟁력을 지니게 된 배경과 주요 원천은 무엇인지 설명해 주세요.


In [15]:
# 다중 쿼리 검색기 생성
multi_query_custom_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 retriever
    llm_chain=multiquery_chain,   # 멀티쿼리 체인
    parser_key="lines"            # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_custom_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments

In [16]:
multi_query_results = evaluate_with_ranx_similarity(
    retriever=multi_query_custom_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

🔍 Starting similarity-based ranx evaluation | 유사도 기반 ranx 평가 시작
   Method | 방법: kiwi_rouge, Threshold | 임계값: 0.8, Mode | 모드: reference_based
   📊 Reference-based mode: Will calculate proper recall metrics | 참조 기반 모드: 정확한 재현율 계산

🔍 Question | 질문 1: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.800
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 질문 2: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.930
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 

ranx similarity evaluation | ranx 유사도 평가: 100%|██████████| 50/50 [02:26<00:00,  3.11s/it]


📊 Similarity-based ranx evaluation results | 유사도 기반 ranx 평가 결과 (kiwi_rouge):
  hit_rate@5: 0.880
  ndcg@5: 0.739
  map@5: 0.679
  mrr: 0.706

📈 Analysis information | 분석 정보:
  Total queries | 총 질문 수: 50
  Total retrieved docs | 총 검색 문서 수: 250
  Avg docs per query | 질문당 평균 검색 문서: 5.0
  Total relevant docs | 관련 문서 총 개수: 76
  Avg relevant per query | 질문당 평균 관련 문서: 1.5
  Threshold used | 사용된 임계값: 0.8
  Overall recall | 전체 재현율: 0.763 (58/76)


### 3) **Decomposition** 

- **단계별 분해 전략**을 통해 복잡한 질문을 작은 단위로 나누어 처리함
- 각 하위 질문마다 **독립적인 검색 프로세스**를 진행하여 정확도를 향상시킴
- **LEAST-TO-MOST PROMPTING**을 활용하여 체계적인 문제 해결 방식을 구현함
- 복잡한 문제를 단순화하여 검색 효율성을 극대화하는 방법론

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_decomposition.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2205.10625

In [17]:
from langchain.prompts import PromptTemplate
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.

    [Input question] 
    {question}

    [Sub-questions (5)]
    """,
)

# 쿼리 생성 체인
decomposition_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = decomposition_chain.invoke({"question": query})

print("생성된 서브 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 서브 질문들:
1. 1. 리비안의 주요 사업 모델과 제품은 무엇인가요?
2. 2. 리비안이 가진 기술적 강점이나 혁신 요소는 무엇인가요?
3. 3. 리비안이 경쟁사와 차별화되는 전략은 무엇인가요?
4. 4. 리비안이 시장에서 현재 차지하고 있는 위치와 성과는 어떠한가요?
5. 5. 리비안이 앞으로 사업 경쟁력을 강화하기 위해 계획 중인 전략이나 목표는 무엇인가요?


In [18]:
# 다중 쿼리 검색기 생성
multi_query_decompostion_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,    # 기본 retriever
    llm_chain=decomposition_chain,   # 서브 질문 생성 체인
    parser_key="lines"               # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_decompostion_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments

In [19]:
decomposition_results = evaluate_with_ranx_similarity(
    retriever=multi_query_decompostion_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

🔍 Starting similarity-based ranx evaluation | 유사도 기반 ranx 평가 시작
   Method | 방법: kiwi_rouge, Threshold | 임계값: 0.8, Mode | 모드: reference_based
   📊 Reference-based mode: Will calculate proper recall metrics | 참조 기반 모드: 정확한 재현율 계산

🔍 Question | 질문 1: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.800
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 질문 2: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.930
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 

ranx similarity evaluation | ranx 유사도 평가: 100%|██████████| 50/50 [03:50<00:00,  4.91s/it]


📊 Similarity-based ranx evaluation results | 유사도 기반 ranx 평가 결과 (kiwi_rouge):
  hit_rate@5: 0.820
  ndcg@5: 0.693
  map@5: 0.644
  mrr: 0.653

📈 Analysis information | 분석 정보:
  Total queries | 총 질문 수: 50
  Total retrieved docs | 총 검색 문서 수: 250
  Avg docs per query | 질문당 평균 검색 문서: 5.0
  Total relevant docs | 관련 문서 총 개수: 76
  Avg relevant per query | 질문당 평균 관련 문서: 1.5
  Threshold used | 사용된 임계값: 0.8
  Overall recall | 전체 재현율: 0.671 (51/76)


### 4) **Step-Back Prompting**

- **단계적 후퇴 방식**을 통해 구체적 질문을 일반적 맥락에서 접근함
- **맥락 기반 검색**으로 넓은 관점에서 구체적 답변으로 좁혀나감
- **포괄적 접근법**을 활용하여 복잡한 질문에 대한 이해도를 높임
- 일반적 맥락에서 시작하여 구체적 해답을 찾아가는 체계적 접근 방식

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_stepback.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2310.06117

`(1) Step-Back 질문 생성`

In [20]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few Shot 예제 - (구체적 질문, 포괄적 질문) 쌍
examples = [
    {
        "input": "애플의 M1 칩 개발이 기업 가치에 미친 영향은?",
        "output": "기업의 핵심 기술 내재화가 경쟁우위에 미치는 영향은 무엇인가?",
    },
    {
        "input": "아마존의 AWS가 수익성에 기여하는 방식은?",
        "output": "기업의 새로운 사업 영역 확장이 수익 구조에 미치는 영향은 무엇인가?",
    },
    {
        "input": "토요타의 하이브리드 기술 전략의 핵심은?",
        "output": "자동차 산업에서 친환경 기술 혁신이 기업 성장에 미치는 영향은 무엇인가?",
    }
]

# 프롬프트 템플릿 초기화
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Step-Back 생성을 위한 프롬프트
step_back_prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """당신은 기업 분석 전문가입니다. 특정 기업에 대한 구체적인 질문을 해당 산업이나 비즈니스 전반의 일반적인 관점에서 
                재해석하는 것이 임무입니다. 산업 동향, 경쟁 구도, 기술 혁신, 사업 모델 등의 관점에서 더 포괄적인 질문으로 
                바꾸어 주세요. 다음은 예시입니다:"""
            ),
            few_shot_prompt,
            ("user", "{question}"),
        ])

# Step-Back 체인 생성
step_back_chain = step_back_prompt | llm | StrOutputParser()

# Step-Back 질문 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
step_back_question = step_back_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"Step-Back 질문: {step_back_question}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
Step-Back 질문: 신생 전기차 기업들이 기존 완성차 업체들과 차별화하여 경쟁우위를 확보하는 전략적 요인은 무엇인가?


In [21]:
# Step-Back 검색기 생성
step_back_retriever = step_back_chain | chroma_k_retriever

# Step-Back 검색 실행
retrieved_docs = step_back_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.

## 기술

### 배터리

Tesla는 CATL, LG Energy Solution 및 Panasonic에서 공급받은 원통형 및 각형 배터리 셀을 사용하고 있으며 자체 배터리를 생산할 수 있는 능력을 구축하고 있습니다.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 제품/서비스를 설계, 제조 및 판매합니다. 2003년 7월 Martin Eberhard와 Marc Tarpenning이 Tesla Motors로 설립했으며, Nikola Tesla를 기리기 위해 명명되었습니다. Elon Musk는 2004년 Tesla의 초기 자금 조달을 주도하여 2008년에 회장 겸 CEO가 되었습니다.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스

In [22]:
stepback_results = evaluate_with_ranx_similarity(
    retriever=step_back_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

🔍 Starting similarity-based ranx evaluation | 유사도 기반 ranx 평가 시작
   Method | 방법: kiwi_rouge, Threshold | 임계값: 0.8, Mode | 모드: reference_based
   📊 Reference-based mode: Will calculate proper recall metrics | 참조 기반 모드: 정확한 재현율 계산

🔍 Question | 질문 1: Rivian Automotive, Inc.의 설립 연도와 주요 사업 분야는 무엇입니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.800
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 1/1
📋 False positives | 거짓 긍정: 4/5
--------------------------------------------------

🔍 Question | 질문 2: 168억 미국 달러는 리브니의 무슨 항목에 해당합니까?...
📊 Reference docs | 참조 문서: 1, Retrieved docs | 검색 문서: 5
📊 Similarity matrix shape | 유사도 매트릭스 shape: (1, 5)
📈 Max similarity | 최대 유사도: 0.175
📋 Qrels items | qrels 항목 수: 1
📋 Run items | run 항목 수: 5
📋 Reference docs found | 참조 문서 발견: 0/1
📋 False positives | 거짓 긍정: 5/5
--------------------------------------------------

🔍 Question | 

ranx similarity evaluation | ranx 유사도 평가: 100%|██████████| 50/50 [01:06<00:00,  1.41s/it]


📊 Similarity-based ranx evaluation results | 유사도 기반 ranx 평가 결과 (kiwi_rouge):
  hit_rate@5: 0.460
  ndcg@5: 0.366
  map@5: 0.335
  mrr: 0.335

📈 Analysis information | 분석 정보:
  Total queries | 총 질문 수: 50
  Total retrieved docs | 총 검색 문서 수: 250
  Avg docs per query | 질문당 평균 검색 문서: 5.0
  Total relevant docs | 관련 문서 총 개수: 76
  Avg relevant per query | 질문당 평균 관련 문서: 1.5
  Threshold used | 사용된 임계값: 0.8
  Overall recall | 전체 재현율: 0.303 (23/76)


`(2) 최종 답변 생성`

In [23]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 초기화
response_prompt = ChatPromptTemplate.from_template(
            """당신은 전문가입니다. 다음 컨텍스트와 질문을 바탕으로 포괄적인 답변을 제공해주세요.

            일반 컨텍스트:
            {normal_context}
            
            기본 개념 컨텍스트:
            {step_back_context}
            
            원래 질문: {question}
            
            답변:"""
        )

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 답변 생성 체인
answer_chain = (
            {
                "normal_context": chroma_k_retriever,
                "step_back_context": step_back_retriever,
                "question": RunnablePassthrough(),
            }
            | response_prompt
            | llm
            | StrOutputParser()
        )

# 답변 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
answer = answer_chain.invoke(query)

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안(Rivian)의 사업 경쟁력은 다음과 같은 주요 요소들에서 기인합니다.

1. **전기차(Pickup 및 SUV) 전문화 및 신시장 개척**  
   리비안은 2021년에 R1T 전기 픽업 트럭 배송을 시작하며, 완전 전기 픽업을 소비자 시장에 본격 출시한 선구자 중 하나입니다. 미국 시장에서 전기 픽업 및 SUV 분야에 집중하여 틈새 시장을 공략하는 전략은 경쟁사 대비 차별화된 포지셔닝을 제공합니다.

2. **통합된 제품 및 서비스 생태계**  
   리비안은 전기자동차뿐만 아니라 배터리 생산, 전기차 충전 서비스, 자동차 보험 등 종합적인 서비스를 함께 제공합니다. 이를 통해 고객에게 더 나은 사용자 경험과 편리성을 제공하며, 충전 인프라 및 보험 서비스까지 아우르는 사업모델로 경쟁력을 가집니다.

3. **전략적 파트너십과 고객 확보**  
   아마존(Amazon)과의 협력관계는 리비안의 중요한 경쟁력입니다. 아마존과 독점 계약을 통해 상업용 전기 배송 밴 공급을 맡았고, 이후 독점 계약 종료 후 다른 상업 고객 대상으로 공급 확대가 가능해져 사업 확장에 유리한 위치를 차지하고 있습니다. 또한 유명 인사나 단체와의 파트너십(예: Alex Honnold, Ewan McGregor 등)을 통해 브랜드 인지도와 신뢰를 강화하고 있습니다.

4. **글로벌 생산 및 연구개발 인프라**  
   미국 내 여러 거점(캘리포니아 어바인 본사, 일리노이 노멀 공장, 미시간 플리머스 엔지니어링 센터 등)에서 차량 설계부터 생산, 공급망 관리 및 소프트웨어 개발까지 수직통합적으로 수행하여 품질 관리 및 생산 효율성을 높이고 있습니다. 특히 소프트웨어 역량 강화를 위한 팰로앨토 오피스와 해외 사무소 확보도 기술 경쟁력에 기여합니다.

5. **혁신적인 기술 및 제품 개발**  
   2023년에는 Tesla 북미 충전 시스템(NACS) 채택과 ABRP 앱 개발사 인수를 통해 고객 충전 편의성을 강화했습니다. 

### 5) **HyDE** (Hypothetical Document Embedding)

- **가상 문서 생성**을 통해 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
- 생성된 문서의 **임베딩 기반 검색**으로 실제 문서와 매칭을 수행함
- **맥락 기반 검색 방식**으로 질문의 의도를 더 정확하게 반영함

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_HyDE.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2212.10496

`(1) 가상 문서 생성`

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# HyDE를 위한 프롬프트 템플릿 생성
template = """주어진 질문에 대한 이상적인 문서 내용을 생성해주세요.
문서는 학술적이고 전문적인 톤으로 작성되어야 합니다.

질문: {question}

문서 내용:"""

hyde_prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 초기화
hyde_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 생성 체인 생성
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

# 문서 생성 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
hypothetical_doc = hyde_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"문서 내용: {hypothetical_doc}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
문서 내용: 리비안(Rivian)의 사업 경쟁력은 다각적인 요소에서 기인하며, 이는 전기차(EV) 시장 내에서의 차별화와 지속 가능한 성장 가능성을 뒷받침하는 핵심 동력으로 작용한다. 본 문서에서는 리비안의 경쟁력을 구성하는 주요 요인들을 기술적 혁신, 제품 포트폴리오, 공급망 관리, 브랜드 전략, 그리고 친환경 가치 실현 측면에서 분석하고자 한다.

1. 기술적 혁신과 플랫폼 경쟁력  
리비안은 독자적인 전기차 플랫폼 ‘스케이트보드(Skateboard)’ 아키텍처를 개발하여, 배터리 팩, 전기 모터, 서스펜션, 전자제어 시스템 등을 통합한 모듈형 설계를 구현하였다. 이 플랫폼은 다양한 차종에 유연하게 적용 가능하며, 높은 주행 성능과 안전성을 확보하는 동시에 생산 효율성을 극대화한다. 특히, 리비안의 전기 구동 시스템은 고출력 모터와 배터리 관리 기술을 결합하여 장거리 주행과 오프로드 주행 능력을 동시에 충족시키는 점에서 경쟁 우위를 가진다.

2. 차별화된 제품 포트폴리오  
리비안은 전통적인 승용 전기차 시장을 넘어 픽업트럭(R1T)과 SUV(R1S) 등 틈새시장을 공략함으로써 독자적인 시장 지위를 구축하고 있다. 이들 차량은 견고한 내구성과 첨단 운전자 보조 시스템(ADAS)을 갖추어, 아웃도어 활동과 상업용 운송 수요를 동시에 겨냥한다. 또한, 리비안은 전기 상용밴(EV Delivery Van) 사업을 통해 대형 물류 기업과의 전략적 파트너십을 확대하며, B2B 시장에서의 입지를 강화하고 있다.

3. 통합적 공급망 및 생산 역량  
리비안은 배터리 셀부터 완성차 조립에 이르는 수직 통합 공급망을 구축하여 원가 절감과 품질 관리를 동시에 달성하고 있다. 특히, 미국 내 미시간과 일리노이주에 위치한 생산 시설은 최신 자동화 기술을 도입하여 생산 유연성과 신속한 시장 대응력을 확보한다. 또한, 리비안은 주요 원자재 공급업체와의 장기 계약을 통해 배터리 소재 확보에 안정성을 기하고 있으며, 이는 전기차 생산 

`(2) 유사 문서 검색`

In [25]:
# 가상 문서를 기반으로 실제 문서 검색
    
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.

## 기술

### 배터리

Tesla는 CATL, LG Energy Solution 및 Panasonic에서 공급받은 원통형 및 각형 배터리 셀을 사용하고 있으며 자체 배터리를 생산할 수 있는 능력을 구축하고 있습니다.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivian은 IPO를 통해 135

`(3) HyDE 검색기 생성`

In [26]:
hyde_retriever = hyde_chain | chroma_k_retriever

retrieved_docs = hyde_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.

## 기술

### 배터리

Tesla는 CATL, LG Energy Solution 및 Panasonic에서 공급받은 원통형 및 각형 배터리 셀을 사용하고 있으며 자체 배터리를 생산할 수 있는 능력을 구축하고 있습니다.
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 'unknown'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 

`(3) 최종 답변 생성`

In [27]:
# 최종 RAG를 위한 프롬프트 템플릿 생성
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {question}

답변:"""

rag_prompt =  ChatPromptTemplate.from_template(template)

# RAG 체인 생성
rag_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
rag_chain = rag_prompt | rag_llm | StrOutputParser()
    
# RAG 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
context = format_docs(retrieved_docs)

answer = rag_chain.invoke({"context": context, "question": query})

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안(Rivian)의 사업 경쟁력은 다음과 같은 요소들에서 비롯됩니다:

1. **전기 픽업 및 SUV 시장 선도**  
   리비안은 2021년 9월 R1T 전기 픽업 트럭을 처음으로 소비자 시장에 출시하며 완전 전기 픽업 분야에서 선도적인 위치를 차지했습니다. 이는 경쟁사 대비 차별화된 제품 포지셔닝을 가능하게 합니다.

2. **강력한 투자 및 자본력**  
   2021년 IPO를 통해 135억 달러를 조달하며 충분한 자본을 확보했고, Amazon, T. Rowe Price, Vanguard, BlackRock, Fidelity Investments 등 주요 기관 투자자들이 최대 주주로 참여해 재무적 안정성과 신뢰를 확보하고 있습니다.

3. **다양한 협력 및 파트너십**  
   Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor 등 다양한 인물 및 단체와의 파트너십을 통해 브랜드 인지도와 사회적 가치를 높이고 있습니다.

4. **북미 중심의 생산 및 엔지니어링 인프라**  
   캘리포니아 어바인 본사에서 차량 엔지니어링 및 설계를 담당하고, 일리노이와 미시간에 제조 및 프로토타입 제작 시설을 운영하며 북미 시장에 최적화된 생산 체계를 갖추고 있습니다.

5. **전기차 충전 및 보험 서비스 제공**  
   단순히 차량 판매에 그치지 않고 전기차 충전 인프라와 자동차 보험 서비스까지 제공함으로써 고객 경험을 확장하고, 수익 다각화를 도모하고 있습니다.

6. **혁신적인 제품 개발과 시장 대응력**  
   부품 비용 상승에 따른 가격 인상과 인력 감축 등 시장 변화에 신속히 대응하며 사업의 지속 가능성을 확보하려는 노력을 보이고 있습니다.

이러한 요소들이 결합되어 리비안은 전기차 시장에서 경쟁력을 갖추고 있으며, 특히 전기 픽업 및 SUV 분야에서 차별화된 입지를 구축하고 있습니다.


`(4) HyDE 체인 종합`

In [28]:
# Step 1. 가상 문서 생성 및 검색
query = "테슬라의 경영진을 분석해주세요."

# Step 2. 유사 문서 검색
retrieved_docs = hyde_retriever.invoke({"question": query})

# Step 3. 최종 답변 생성
final_answer = rag_chain.invoke(
    {
        "context": format_docs(retrieved_docs), 
        "question": query
    }
)

print(f"쿼리: {query}")
print(f"가상 문서 내용: {hypothetical_doc}")
print(f"답변: {final_answer}")

쿼리: 테슬라의 경영진을 분석해주세요.
가상 문서 내용: 리비안(Rivian)의 사업 경쟁력은 다각적인 요소에서 기인하며, 이는 전기차(EV) 시장 내에서의 차별화와 지속 가능한 성장 가능성을 뒷받침하는 핵심 동력으로 작용한다. 본 문서에서는 리비안의 경쟁력을 구성하는 주요 요인들을 기술적 혁신, 제품 포트폴리오, 공급망 관리, 브랜드 전략, 그리고 친환경 가치 실현 측면에서 분석하고자 한다.

1. 기술적 혁신과 플랫폼 경쟁력  
리비안은 독자적인 전기차 플랫폼 ‘스케이트보드(Skateboard)’ 아키텍처를 개발하여, 배터리 팩, 전기 모터, 서스펜션, 전자제어 시스템 등을 통합한 모듈형 설계를 구현하였다. 이 플랫폼은 다양한 차종에 유연하게 적용 가능하며, 높은 주행 성능과 안전성을 확보하는 동시에 생산 효율성을 극대화한다. 특히, 리비안의 전기 구동 시스템은 고출력 모터와 배터리 관리 기술을 결합하여 장거리 주행과 오프로드 주행 능력을 동시에 충족시키는 점에서 경쟁 우위를 가진다.

2. 차별화된 제품 포트폴리오  
리비안은 전통적인 승용 전기차 시장을 넘어 픽업트럭(R1T)과 SUV(R1S) 등 틈새시장을 공략함으로써 독자적인 시장 지위를 구축하고 있다. 이들 차량은 견고한 내구성과 첨단 운전자 보조 시스템(ADAS)을 갖추어, 아웃도어 활동과 상업용 운송 수요를 동시에 겨냥한다. 또한, 리비안은 전기 상용밴(EV Delivery Van) 사업을 통해 대형 물류 기업과의 전략적 파트너십을 확대하며, B2B 시장에서의 입지를 강화하고 있다.

3. 통합적 공급망 및 생산 역량  
리비안은 배터리 셀부터 완성차 조립에 이르는 수직 통합 공급망을 구축하여 원가 절감과 품질 관리를 동시에 달성하고 있다. 특히, 미국 내 미시간과 일리노이주에 위치한 생산 시설은 최신 자동화 기술을 도입하여 생산 유연성과 신속한 시장 대응력을 확보한다. 또한, 리비안은 주요 원자재 공급업체와의 장기 계약을 통해 배터리 소재 확보에 안정성을 기하고 있으며, 이는 전기차 생산 확대

# 정리

쿼리확장

| 방법론 | 핵심 아이디어 | 장점 | 단점 | 적용 상황 |
|--------|---------------|------|------|-----------|
| **Query Reformulation** | LLM으로 질문 재작성 | 구현 간단, 즉시 적용 | 단일 변형만 생성 | 일반적인 질문 개선 |
| **Multi Query** | 다양한 관점의 질문 생성 | 검색 다양성 증가 | 계산 비용 증가 | 모호한 질문 처리 |
| **Decomposition** | 복잡한 질문을 하위 질문으로 분해 | 체계적 접근 | 분해 정확도 의존 | 복합적 질문 |
| **Step-Back Prompting** | 일반적 맥락에서 구체적 답변으로 | 포괄적 이해 | 추가 검색 필요 | 전문적/복잡한 질문 |
| **HyDE** | 가상 답변 문서 생성 | 의미적 정렬 우수 | 환각 위험 | Zero-shot 상황 |

In [ ]:
######## 준비 단계 ########
# 1 벡터 스토어 로드
def initialize_vector_store(embeddings = OpenAIEmbeddings(model="text-embedding-3-small"), collection_name="hybrid_search_db", persist_directory = "./local_chroma_db"):
    """
    기존 벡터 저장소를 로드하거나 새로 생성
    
    Returns:
        Chroma: 벡터 저장소 객체
    """
    try:
        
        # 기존 벡터 저장소 로드 시도
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory,
        )
        
        doc_count = vector_store._collection.count()
        if doc_count > 0:
            print(f"✅ 기존 벡터 저장소 로드: {doc_count}개 문서")
            return vector_store
        else:
            print("⚠️ 빈 벡터 저장소입니다. 데이터를 추가해주세요.")
            return vector_store
            
    except Exception as e:
        print(f"❌ 벡터 저장소 로드 실패: {e}")
        return None
chroma_db = initialize_vector_store()


# 2 백터 검색기 생성 (chroma_k_retriever)
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = chroma_k_retriever.invoke(query)
# retriever 결과 출력
for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)



######## Query Redormulation ########

# 쿼리 리포뮬레이션을 위한 프롬프트 템플릿 정의
reformulation_template = """다음 질문을 검색 성능을 향상시키기 위해 다시 작성해주세요:
[질문]
{question}

다음 방식으로 질문을 재작성하세요:
1. 동의어 추가
2. 더 구체적인 키워드 포함
3. 관련된 개념 확장

[재작성된 질문]
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(reformulation_template)

# LLM 모델 초기화
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 쿼리 리포뮬레이션 체인 생성
reformulation_chain = prompt | llm | StrOutputParser()

# 체인 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
reformulated_query = reformulation_chain.invoke({"question": query})


# 111 리포뮬레이션된 쿼리로 검색 래보기
retrieved_docs = chroma_k_retriever.invoke(reformulated_query)
for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)


# Runnable 객체로 변환하여 검색기 생성 (LCEL)
reformulation_retriever = reformulation_chain | chroma_k_retriever

# 222 쿼리 리포뮬레이션 검색기 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = reformulation_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)


######## 평가 ########
def load_evaluation_dataset(file_path):
    """
    평가 데이터셋 로드
    
    Args:
        file_path (str): 평가 데이터 파일 경로
    
    Returns:
        pandas.DataFrame: 평가 데이터셋
    """
    try:
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            raise ValueError("지원하지 않는 파일 형식")
        
        print(f"✅ 평가 데이터셋 로드: {len(df)}개 질문")
        return df
    except Exception as e:
        print(f"❌ 데이터셋 로드 실패: {e}")
        return None

# 평가 데이터셋 로드

eval_df = load_evaluation_dataset("./data/synthetic_testset.csv")
def prepare_evaluation_data(df):
    """
    평가 데이터 전처리
    
    Args:
        df (pandas.DataFrame): 원본 데이터프레임
    
    Returns:
        tuple: (질문 리스트, 정답 문서 리스트)
    """
    questions = df['user_input'].tolist()
    
    # 정답 문서 파싱
    reference_contexts = []
    for contexts in df['reference_contexts']:
        if isinstance(contexts, str):
            # 문자열을 리스트로 변환
            context_list = eval(contexts)
        else:
            context_list = contexts
        
        # Document 객체로 변환
        docs = [Document(page_content=ctx) for ctx in context_list]
        reference_contexts.append(docs)
    
    return questions, reference_contexts

# 평가 데이터 전처리
questions, reference_contexts = prepare_evaluation_data(eval_df)



######## 평가 실행 ######

# ranx-k 평가 실행 (rouge 점수가 높은 경우) -> 문자열 유사도 기반 평가
chroma_results = evaluate_with_ranx_similarity(
    retriever=chroma_k_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)


reformulation_results = evaluate_with_ranx_similarity(
    retriever=reformulation_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)



#####멀티 쿼리 #####

# LLM 모델 초기화 (멀티 쿼리 생성용)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.7,
)

# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = multi_query_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)